<b>For use with: <font color="red" >Alpaca Kernel</font></b>
<br>
Accessibility of this notebook  

    
>- Adjust the width of the text with the function provided in the cell below. 
>- Toggle the auto-numbering of the sections in the outline toolbox (sidebar or topbar).
>- Toggle the code line numbers in the dropdown menu of the "view" button in the topbar. 
>- Collapse/expand a cell by clicking the blue bar on the left side of the cell.



In [ ]:
%python 
from IPython.core.display import HTML
def set_width(width):
    display(HTML(f"""<style>  
            .container {{ width:{width}% !important; 
                            min-width:800px !important; margin: 0 auto}} 
            .jp-Cell {{ width:{width}% !important; 
                            min-width:800px !important; margin: 0 auto}} </style>"""))
# Set container width to X%v of the fullscreen 
set_width(50)


# 17C - Implement and Investigate: Linear Potentiostat Model with Alpaca


## Goals

>1. Build a simple model of the potentiostat on Alpaca
>2. Get an intuition for the (multiple) inversion of polarity between potentiostat's control and readout</br>
>3. Compute `Icell` from the measurements, and compose you first voltammogram</br>
>4. Optional: Compute the range and accuracy of `Icell`</br>
>5. Optional: Suppress the noise

## Implement 1: Implement the linear Potentiostat Model

It's up to your decision whether you will use the Basic or the Advanced Design to implement the resistor-based Measurement Cell Model 1. 


### Design References

<details>
<summary><font color='darkblue' size=4><b>Schematic: Basic Design - Model 1</b></font></summary>

<div style="text-align: center">
    <img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/DesignBModel1+ADC.png" width=1000>
    <br>
    <em>Design Basic: Model 1 implementation on Alpaca</em>
</div><br>
</details> 
<br>

<details>
<summary><font color='darkblue' size=4><b>Schematic: Advanced Design - Model 1</b></font></summary>
<div style="text-align: center">
    <img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/DesignAModel1+ADC.png" width=1000>
    <br>
    <em>Design Basic: Model 1 implementation on Alpaca</em>
</div><br>
</details> 
<br>

<details>
<summary><font color='purple' size=4><b>Fritzing: Alpaca's default jumpers</b></font></summary>
<div style="text-align: center">
    <img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/17_Fritzing_Alpaca_Check_Jumpers.png" width=1000>
    <br>
    <em>Alpaca - The starting point configuration for Voltammetry</em>
</div><br>
</details> 
<br>

<details>
<summary><font color='purple' size=4><b>Fritzing: Basic Design - Barebone with Relay</b></font></summary><br>
<div style="text-align: center">
    <img src="https://gitlab.tudelft.nl/mwdocter/nb2214-images/-/raw/main/voltammetry/2024/17C_Fritzing_B_Barebone_with_Relay.png" width=1000>
    <br>
    <em>Basic Design: Barebone with Relay</em>
</div><br>
</details> 
<br>

## Implement and Investigate 2.1: $U_{\text{DAC}}$ and $U_{\text{ADC}}$


<font color='brown' size=3><b>Recall or derive the formula for $U_{ADC}$ as a function of $U_{\text{DAC}}$ for the chosen design.</b></font> </br>
Look up your results from 17A, and check Udac -> Udaq_assistant -> Ucell -> Ua -> Uadc. First we will noly look at Udac -> Uadc.

Take the relevant parameters into account, write a code to set a desired `Udac` and to measure `Uadc`. 
> **Basic design**<br>
> - `Udac_assistant = function of UdacA`
> - `Uadc = Uadc0` and/or `Uadc=Uadc1` 
>   
> **Advanced design**
> - `Udac = UdacA` and/or `Udac = UdacB`
> - `Uadc = Uadc0` <br>

Use resistors `R1=10k`, `R2=10k` and `Rf=10k` and set the attenuation for the ADCs to `1:1`. 

1. Set `Udac = 1V` and  `Udac = -1V`, and measure `Uadc`.
2. Convert the measured `Uadc` to Volts and compare with the expected outcome. Pay attention to the correct sign in your results (polarity).

<details>
<summary><font size=3>💡<b>Hints: Custom functions</b></font></summary>

> You may use functions included in `picotools` as a reminder on how to control the DAC and the readouts.
>
> However, we recommend that you write your own functions, because in practice, adapting the existing unfamiliar functions for this purpose might take longer than writing your own.
>
> **Optional:** Consider creating your own Python module for this project, for example: `picovolt.py`
> and import your functions with, for example: <br>
> ```python
> import picovolt as pv
> data = pv.function(param1, param2, ...)
>```
</details> 
<br>   

3. Write a code to loop over the `Udac` and to store the measured output voltages.

> **Basic Design** <br>
> Perform a voltage sweep from `Udac=-0.5V` to `Udac=1.5V` and plot the measured `Uadc`
>
> **Advanced Design**<br>
> Perform a voltage sweep from `UdacA=0V` to `UdacA=2V` and plot the measured `Uadc`
> 
> - while `UdacB=0V`,
> - while `UdacB=1V`
> 
> <br>
>
> *Optional: Swap the parameters for `UdacA` with `UdacB`*

<details><summary><font size=3>💡<b>Hint: Function Generator</b></font></summary> 
    
> You may in this assignment use the in-built Function Generator to extract a `Triangle` waveform with a required symmetry. <br>
>
>
> ```python
> from functiongenerator import Triangle
>
> triangle = Triangle(Vmin=V_min, Vmax=V_max, freq=1, symmetry=50) 
> waveform = triangle.get_voltages(n=NUM_SAMPLES)
>
> for ii in range(NUM_SAMPLES):
>     dac_a.write(waveform[ii])
>```

</details> 
<br>

### Conclude
With help of the plots, reflect on:
  > *Goal 2: Get an intuition for the (multiple) inversion of polarity between potentiostat's control and readout* </br>

## Implement and Investigate 2.2: Cell Potential

**Recall or derive the formula for $U_{a}$ as a function of $U_{\text{cell}}$ in Model 1.** 
Now we dive into the voltages between Udac & Uadc: Udac -> Udaq_assistant -> Ucell -> Ua -> Uadc.

Use the experience from the previous section, and write a code that
- sets `Udac` to generate the desired `Ucell`
- and computes `Ua` from the measurement of `Uadc`.

Then, 
1. Run a measurement for `Ucell = 1V` and `Ucell = -1V` and extract the `Ua`.
   - **Optional:** Measure the actual `Ucell` with `Ain2` as a reference signal for later.

<font color='black' size=3>💡<b> Hints</b></font>
> - Remember to *reset* your Pico before the measurement and save the data
> - Choose to measure with or without `-12V` to Cria

2. Compare the measured `Ua` with the expected outcome.

3. Write a code to loop over the `Ucell` and to store the measured output voltages.

4. Perform a voltage sweep `Ucell=-1V` to `Ucell=1V` and convert the measured signal to `Ua`

5. Plot the measured `Ua` along with a calculated, expected `Ua_ref` and discuss any potential differences in each case.


6. Save your results in a file and fetch it from Pico to your computer

## Implement and Investigate 3: Cell Current 

**Recall or derive the formula for $I_{\text{cell}}$ as a function of $U_{\text{cell}}$ for Model 1 in your design.** 

1. Extract `Icell` from the measurement of `Ua` saved in *Implement and Investigate 3: Cell Potential*.

2. Plot `Icell` as a function of `Ucell`.

3. **Optional:** Include a suitable capacitor in your circuit to reduce the noise. Compute the cut-off frequency.

4. **Optional:** Compute the accuracy and the noise in the measured `Icell` to evaluate the effect of the Noise filter.

5. **Optional:** Replace the resistors with `R1=R2=Rf=1k`. Then, run the same measurement and plot `Icell` as a function of `Ucell`. Finally, compare the quality of this plot with the one recorded with the previous configuration. 

---
# Compare and Conclude

> <font color='grey'>⏳ Estimated time: 5 min</font>

## To be checked off by a TA

1. Present your plot of `Icell` over `Ucell`.
2. Discuss your results from the optional tasks.

---

## Optional - Implement and Investigate 4: Progress to Week 18

### I4.1: Optimise the range of detectable `Icell` to improve the Signal-to-Noise ratio.

**Recall or derive the formula for the range of `Icell` as a function of all relevant parameters in Model 1 for your chosen design.**

>**Goal:** Find out how to configure your Potentiostat to ensure that are making use of the avaialable limits to record the strongest signal within a predefined range of `Icell`. 

1. Compute the range of possible `Icell` as a function `Rf` (and in the Advanced Design as a function of `Uoffset`) for roughly the entire range of possible `Ua` with the given configuration of resistors.
   > the range of `Ua` is the range of the possible outputs of `OPAMP2`: <br>
   > Basic - positive and negative, <br>
   > Advanced - only positive.

2. Select `Rf` to increase the current-to-voltage conversion factor and to utilise *roughly* the entire range of `Ua` without causing clipping on `OPAMP2`, and adjust the attenuation of `AMPs` to capture the signal from `Ua` without *clipping* the `ADCs`.

    > Note that it could be optimal to slightly re-adjust the value of `Rf` because there are only a few available options of attenuation factors. It is also fine to leave some margins. 
 
   **Advanced Design:** Focus on optimising the range for the positive currents first, so do not change the `Uoffset` yet. Notice what effect the change of `Rf` has on the `Icell` range for the negative currents.

3. **Advanced Design:** Derive a formula for setting a desired range of detectable `Icell` and describe the method to adjust that range while maintaining a fixed range of `Ucell`.
   > Consider writing a function to compute that `Icell` range for a given set of parameters or to compute the required parameters for a given range of currents.

4. Pre-compute several essential `Icell` ranges for a pre-defined set of ranges of `Ucell`.
   > **Optional:** Write a function to compute the required parameters on demand.

### I4.2: Noise reduction

#### Filtering

Figure out how to estimate the bound for a suitable the cut-off frequency of your filter that will only negligibly affect your Voltammetry measurements.

#### Averaging

Gather ideas on how to write an optimised code for measurements with averaging to reduce its effect on the timing precision.